### Extracción de indicadores del Sistema de Indicadores de la Ciudad de México

Este script extrae los indicadores del Sistema de Indicadores de la Ciudad de México. La información se obtiene de la página web https://indicadores.cdmx.gob.mx/dataset/. Se extraen los siguientes campos:
- Título
- Descripción
- Instrumento
- Derecho
- ODS
- Fuente
- URL de la fuente

In [21]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36'}

In [46]:
# Initialize lists to store the data
titulos = []
descripcion = []
instrumento = []
derecho = []
ods = []
fuente = []
fuente_url = []

# Loop de 33 páginas
for page in range(1, 34):
    url_basica = f"https://indicadores.cdmx.gob.mx/dataset/?page={page}"

    # Petición a la página y parseo con BeautifulSoup
    r = requests.get(url_basica, headers=headers, verify=False)
    sopa = BeautifulSoup(r.text, "html.parser")

    # Objetivo de la extracción
    resultados = sopa.find_all("div", class_="col-sm-12")

    # Extraer los datos de cada indicador
    for datos in resultados:
        # Título
        h2_tag = datos.find('h2', class_='cp-section-heading cp-default-section-heading cp-section-anchored-heading')
        if h2_tag:
            cleaned_text = h2_tag.text.strip()
            titulos.append(cleaned_text)

        # Descripción
        p_tag = datos.find('p')
        if p_tag:
            cleaned_text = p_tag.text.strip()
            descripcion.append(cleaned_text)

        #instrumento, derecho, ods
        a_tags = datos.find_all('a')
        if len(a_tags) >= 1:
            instrumento.append(a_tags[0].text.strip())
        if len(a_tags) >= 2:
            derecho.append(a_tags[1].text.strip())
        if len(a_tags) >= 3:
            ods.append(a_tags[2].text.strip())

        # Fuente y URL de la fuente
        fuente_tag = datos.find('div', id='fuente')
        if fuente_tag:
            fuente_text = fuente_tag.find('a')
            if fuente_text:
                fuente.append(fuente_text.text.strip())
                fuente_url.append(fuente_text['href'])

C:\Users\claud\miniconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'indicadores.cdmx.gob.mx'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\claud\miniconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'indicadores.cdmx.gob.mx'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\claud\miniconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'indicadores.cdmx.gob.mx'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\claud\miniconda3\Lib\s

In [47]:
# Crea tabla
tabla_indicadores = pd.DataFrame({
    "titulo": titulos,
    "descripcion": descripcion,
    "instrumento": instrumento,
    "derecho": derecho,
    "ods": ods,
    "fuente": fuente,
    "fuente_url": fuente_url
})

In [50]:
#Eliminar duplicados de nombre
tabla_indicadores = tabla_indicadores.drop_duplicates(subset='titulo')
tabla_indicadores

,titulo,descripcion,instrumento,derecho,ods,fuente,fuente_url
0,Número de incendios forestales,Total de incendios forestales al año.,PGOT,Derecho a un medio ambiente sano,Acción por el clima,"CONAFOR, Gerencia del Manejo del Fuego",http://dgeiawf.semarnat.gob.mx:8080/ibi_apps/W...
2,Número de solicitudes de patentes realizadas p...,Total de solicitudes de patentes que fueron re...,PGD,Derecho a la ciencia y a la innovación tecnoló...,"Industria, innovación e infraestructura",IMPI,https://www.gob.mx/impi/documentos/instituto-m...
3,Cifra negra de delitos,Cifra negra: delitos no denunciados; delitos d...,PGD,Derecho de acceso a la justicia,"Paz, justicia e instituciones sólidas","INEGI, Encuesta Nacional de Victimización y Pe...",https://www.inegi.org.mx/programas/envipe/2023/
4,Porcentaje de población de 15 años y más en re...,Porcentaje de la población de 15 años y más en...,PGD,Derecho a la educación,Educación de calidad,Estimación del rezago educativo,https://www.gob.mx/inea/documentos/rezago-educ...
5,Empleos en el IMSS,Indicador mensual de personas registradas como...,Visor CDMX,Derecho al trabajo,Trabajo decente y crecimiento económico,Instituto Mexicano del Seguro Social,https://public.tableau.com/app/profile/imss.cp...
...,...,...,...,...,...,...,...
675,Miembros del Sistema de Divulgación y Apropiac...,Número de miembros del Sistema de Divulgación ...,PDCTI,Derecho a la ciencia y a la innovación tecnoló...,Educación de calidad,"SECTEI; , CONACYT",https://www.sectei.cdmx.gob.mx/
676,Porcentaje de entidades y dependencias con pro...,"En el 2020, 8 entidades y dependencias tienen ...",PDCTI,Derecho a la ciencia y a la innovación tecnoló...,"Industria, innovación e infraestructura","LOPEAP, APCDMX, Portal de Transparencia de la ...",No disponible
677,Superficie de las colonias con un nivel de equ...,Superficie total de las colonias con un nivel ...,PGOT,Derecho a la Ciudad,"Industria, innovación e infraestructura",CentroGeo,No disponible
678,Porcentaje de empresas que realizaron proyecto...,Porcentaje de empresas que realizaron proyecto...,PDCTI,Derecho a la ciencia y a la innovación tecnoló...,"Industria, innovación e infraestructura","INEGI, Encuesta Sobre Investigación y Desarrol...",https://www.inegi.org.mx/programas/esidet/2017/


In [55]:
#Salvar como csv
tabla_indicadores.to_csv("indicadores_sid_cdmx.csv", index=False, encoding='utf-8-sig')